# 01_scrap

## Getting data from THE STANDARD

In [ ]:
from bs4 import BeautifulSoup as soup
import requests
from datetime import datetime
import pandas as pd
import json

In [ ]:
r = requests.get('https://thestandard.co/?s=ส่งออกทุเรียน&search=')
b = soup(r.content,'html.parser')

In [ ]:
title = []
urls = []
content = []
published_date = []
retrieved_date  = []

In [ ]:
for news in b.findAll('div', attrs={'class' : 'col-md-8'}):
    titles = news.find('h3').get_text()
    titles = titles.replace('\n','')
    urls.append(news.find('a', href=True)['href'])
    #print(titles)
    title.append(titles)

#print(title)
#print(urls)

In [ ]:
for link in urls:
    page = requests.get(link)
    bsobj = soup(page.content,'html.parser')
    for n in bsobj.findAll('div', attrs={'class' : 'entry-content'}):
        dis = n.text
        dis = dis.replace('\n','')
        dis = dis.replace('\xa0','')
        content.append(dis)

    for date in bsobj.findAll('div', attrs={'class' : 'meta-date'}):
        date_p = date.find('span').get_text()
        date_p = datetime.strptime(date_p, '%d.%m.%Y').strftime("%Y-%m-%d")
        published_date.append(date_p)
        
    r = (datetime.now()).strftime("%Y-%m-%d")
    retrieved_date.append(r)
    
#print(content) 
#print(published_datetime)
#print(retrieved_datetime)

In [ ]:
table = pd.DataFrame([title, published_date, retrieved_date, content, urls]).transpose()
table.columns = ['title','publish_date','retrieve_date','content','source_url']
table.head()

In [ ]:
table = table.to_dict('records')

In [ ]:
with open('exporting_durians_the_standard.json','w', encoding='utf8') as json_file:
    json.dump(table, json_file)

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 60

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='exporting_durians_the_standard.json')